In [1]:
import numpy as np
n_clusters = 5
labels = np.random.randint(0, n_clusters, 100)

In [2]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# code for the PAIF manuscript:
# F. von Wegner, Partial Autoinformation to Characterize Symbolic Sequences
# Front Physiol (2018) https://doi.org/10.3389/fphys.2018.01382
# FvW 05/2018, Python3 version 05/2021

import os, sys
import numpy as np
import matplotlib.pyplot as plt

# --- define logarithm
#log = np.log
log = np.log2


"""*****************************************************************************
!!! ALL SYMBOLS HAVE TO BE INTEGERS: 0, 1, 2, ... !!!
*****************************************************************************"""


def H_1(x, ns):
    """
    Shannon entropy of the symbolic sequence x with ns symbols.

    Args:
        x: symbolic sequence, symbols = [0, 1, ..., ns-1]
        ns: number of symbols
    Returns:
        h: Shannon entropy of x
    """

    n = len(x)
    p = np.zeros(ns)  # symbol distribution
    for t in range(n):
        p[x[t]] += 1.0
    p /= n
    h = -np.sum(p[p>0]*log(p[p>0]))
    return h


def H_2(x, y, ns):
    """
    Joint Shannon entropy of the symbolic sequences x, y, with ns symbols.

    Args:
        x, y: symbolic sequences, symbols = [0, 1, ..., ns-1]
        ns: number of symbols
    Returns:
        h: joint Shannon entropy of (x, y)
    """

    if (len(x) != len(y)):
        print("H_2 warning : sequences of different lengths, using the shorter...")
    n = min(len(x), len(y))
    p = np.zeros((ns, ns)) # joint distribution
    for t in range(n):
        p[x[t],y[t]] += 1.0
    p /= n
    h = -np.sum(p[p>0]*log(p[p>0]))
    return h


def H_k(x, ns, k):
    """
    Joint Shannon entropy of k-histories x[t:t+k]

    Args:
        x: symbolic sequence, symbols = [0, 1, ..., ns-1]
        ns: number of symbols
        k: length of k-history
    Returns:
        h: joint Shannon entropy of x[t:t+k]
    """

    n = len(x)
    p = np.zeros(tuple(k*[ns]))  # symbol joint distribution
    for t in range(n-k):
        p[tuple(x[t:t+k])] += 1.0
    p /= (n-k) # normalize distribution
    h = -np.sum(p[p>0]*log(p[p>0]))
    #m = np.sum(p>0)
    #h = h + (m-1)/(2*N) # Miller-Madow bias correction
    return h

def H_k_fix(x, ns, k):
    """
    Joint Shannon entropy of k-histories x[t:t+k]

    Args:
        x: symbolic sequence, symbols = [0, 1, ..., ns-1]
        ns: number of symbols
        k: length of k-history
    Returns:
        h: joint Shannon entropy of x[t:t+k]
    """

    n = len(x)
    p = np.zeros(tuple(k*[ns]))  # symbol joint distribution
    for t in range(n-k+1):
        p[tuple(x[t:t+k])] += 1.0
    p /= (n-k+1) # normalize distribution
    h = -np.sum(p[p>0]*log(p[p>0]))
    #m = np.sum(p>0)
    #h = h + (m-1)/(2*N) # Miller-Madow bias correction
    return h

def ais(x, ns, k=1):
    """
    Active information storage (AIS)

    TeX notation:
    I(X_{n+1} ; X_{n}^{(k)})
    = H(X_{n+1}) - H(X_{n+1} | X_{n}^{(k)})
    = H(X_{n+1}) - H(X_{n+1},X_{n}^{(k)}) + H(X_{n}^{(k)})
    = H(X_{n+1}) + H(X_{n}^{(k)}) - H(X_{n+1}^{(k+1)})

    Args:
        x: symbolic sequence, symbols = [0, 1, ..., ns-1]
        ns: number of symbols
        k: history length (optional, default value k=1)
    Returns:
        a: active information storage
    """

    n = len(x)
    h1 = H_k(x, ns, 1)
    h2 = H_k(x, ns, k)
    h3 = H_k(x, ns, k+1)
    a = h1 + h2 - h3
    return a

def ais_fix(x, ns, k=1):
    """
    Active information storage (AIS)

    TeX notation:
    I(X_{n+1} ; X_{n}^{(k)})
    = H(X_{n+1}) - H(X_{n+1} | X_{n}^{(k)})
    = H(X_{n+1}) - H(X_{n+1},X_{n}^{(k)}) + H(X_{n}^{(k)})
    = H(X_{n+1}) + H(X_{n}^{(k)}) - H(X_{n+1}^{(k+1)})

    Args:
        x: symbolic sequence, symbols = [0, 1, ..., ns-1]
        ns: number of symbols
        k: history length (optional, default value k=1)
    Returns:
        a: active information storage
    """

    n = len(x)
    h1 = H_k_fix(x, ns, 1)
    h2 = H_k_fix(x, ns, k)
    h3 = H_k_fix(x, ns, k+1)
    a = h1 + h2 - h3
    return a


In [3]:
# n_clusters doesn't change H1
for n in [n_clusters, 10, 12]:
    print(H_1(labels, n))

2.2940526443131133
2.2940526443131133
2.2940526443131133


In [4]:
# n_clusters doesn't change H_k
for n in [n_clusters, 10, 12]:
    print(H_k(labels, n, 2))

4.506185519664854
4.506185519664854
4.506185519664854


In [5]:
# H_k (k=1)!= H_1
print(H_1(labels, n))
print(H_k(labels, n, 1))
# due to len(labels) - k + 1
print(H_k_fix(labels, n, 1))

2.2940526443131133
2.290127190985782
2.2940526443131133


In [7]:
%load_ext autoreload
%autoreload 2

from pycrostates.segmentation.autoinformation import entropy,joint_entropy, joint_entropy_history

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
entropy(labels) == H_1(labels, n_clusters)

True

In [14]:
np.allclose(joint_entropy(labels, labels), H_2(labels, labels, n_clusters))

True

In [18]:
np.allclose(joint_entropy_history(labels, 3), H_k_fix(labels, n_clusters, 3))

True

In [16]:
H_k(labels, n_clusters, 3)

6.117903692680288

6.137619155317624